In [248]:
# Import necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# Load the datasets
diabetic_data = pd.read_csv('diabetic_data.csv')
ids_mapping = pd.read_csv('IDS_mapping.csv')

# Display the first few rows of the datasets
print("Diabetic Data:")
print(diabetic_data.head())

print("\nIDS Mapping:")
print(ids_mapping.head())

Diabetic Data:
   encounter_id  patient_nbr             race  gender      age weight  \
0       2278392      8222157        Caucasian  Female   [0-10)      ?   
1        149190     55629189        Caucasian  Female  [10-20)      ?   
2         64410     86047875  AfricanAmerican  Female  [20-30)      ?   
3        500364     82442376        Caucasian    Male  [30-40)      ?   
4         16680     42519267        Caucasian    Male  [40-50)      ?   

   admission_type_id  discharge_disposition_id  admission_source_id  \
0                  6                        25                    1   
1                  1                         1                    7   
2                  1                         1                    7   
3                  1                         1                    7   
4                  1                         1                    7   

   time_in_hospital  ... citoglipton insulin  glyburide-metformin  \
0                 1  ...          No      No      

In [249]:
# Identify and handle non-standard missing values
missing_value_indicators = ['?', 'Unknown/Invalid', 'NULL', 'Not Available', 'Not Mapped', 'NA', 'None']

# Replace missing value indicators with NaN
for indicator in missing_value_indicators:
    diabetic_data.replace(indicator, np.nan, inplace=True)

# Check for missing values in diabetic data
missing_values = diabetic_data.isnull().sum()
print("\nMissing Values in Diabetic Data:")
print(missing_values[missing_values > 0])


Missing Values in Diabetic Data:
race                  2273
gender                   3
weight               98569
payer_code           40256
medical_specialty    49949
diag_1                  21
diag_2                 358
diag_3                1423
max_glu_serum        96420
A1Cresult            84748
dtype: int64


In [250]:
# Remove features with too many missing values
features_to_remove = ['weight', 'payer_code']
diabetic_data.drop(columns=features_to_remove, inplace=True)

# Remove medications which were not prescribed
unique_value_counts = diabetic_data.nunique()
features_to_remove = unique_value_counts[unique_value_counts == 1].index
diabetic_data.drop(columns=features_to_remove, inplace=True)
print("Features removed:")
print(features_to_remove)

# Handle features by filling missing values with 'missing'
diabetic_data.fillna({'medical_specialty': 'NotSpecefied'}, inplace=True)
diabetic_data.fillna({'max_glu_serum': 'TestNotPerformed'}, inplace=True)
diabetic_data.fillna({'A1Cresult': 'TestNotPerformed'}, inplace=True)
diabetic_data.fillna({'race': 'Missing'}, inplace=True)
diabetic_data.dropna(subset=['gender'], inplace=True)
missing_values = diabetic_data.isnull().sum()
print("\nMissing Values in Diabetic Data:")
print(missing_values[missing_values > 0])

Features removed:
Index(['examide', 'citoglipton'], dtype='object')

Missing Values in Diabetic Data:
diag_1      21
diag_2     358
diag_3    1423
dtype: int64


In [251]:
# remove patients prescribed troglitazone for toxicity and withdrawl and remove feature
diabetic_data = diabetic_data[diabetic_data['troglitazone'] == 'No']
diabetic_data.drop(columns=['troglitazone'], inplace=True)

Now we remove the duplicate patient encounters.

In [252]:
diabetic_data = diabetic_data.loc[diabetic_data.groupby('patient_nbr')['encounter_id'].idxmin()]


And then remove hospice and expired patients.

In [253]:
high_risk_dispositions = [11, 13, 14, 19, 20, 21]
diabetic_data = diabetic_data[~diabetic_data['discharge_disposition_id'].isin(high_risk_dispositions)]

Now we prepare for one hot encoding by converting diagnoses codes to disease groups

In [254]:
# Define the diagnosis code mapping with detailed labels and convert to strings
diagnosis_mapping = {
    'Diseases_of_circulatory_system': {str(i) for i in range(390, 460)}.union({'785'}),
    'Diseases_of_the_respiratory_system': {str(i) for i in range(460, 520)}.union({'786'}),
    'Diseases_of_the_digestive_system': {str(i) for i in range(520, 580)}.union({'787'}),
    'Diabetes_mellitus': {'250'},
    'Injury_and_poisoning': {str(i) for i in range(800, 1000)},
    'Diseases_of_the_musculoskeletal_system_and_connective_tissue': {str(i) for i in range(710, 740)},
    'Diseases_of_the_genitourinary_system': {str(i) for i in range(580, 630)}.union({'788'}),
    'Neoplasms': {str(i) for i in range(140, 240)},
    'Other_symptoms_signs_and_ill_defined_conditions': {str(i) for i in range(790, 800)}.union({'780', '781', '784'}),
    'Endocrine_nutritional_and_metabolic_diseases_and_immunity_disorders_without_diabetes': {str(i) for i in range(240, 280) if i != 250},
    'Diseases_of_the_skin_and_subcutaneous_tissue': {str(i) for i in range(680, 710)}.union({'782'}),
    'Infectious_and_parasitic_diseases': {str(i) for i in range(1, 140)},
    'Mental_disorders': {str(i) for i in range(290, 320)},
    'Diseases_of_the_blood_and_blood_forming_organs': {str(i) for i in range(280, 290)},
    'Diseases_of_the_nervous_system': {str(i) for i in range(320, 360)},
    'Complications_of_pregnancy_childbirth_and_the_puerperium': {str(i) for i in range(630, 680)},
    'Diseases_of_the_sense_organs': {str(i) for i in range(360, 390)},
    'Congenital_anomalies': {str(i) for i in range(740, 760)},
    'External_causes_of_injury_and_supplemental_classification': {'E'},
    'Supplementary_classification': {'V'}
}


# Function to map diagnosis codes to categories
def map_diagnosis(code):
    if pd.isna(code) or code == 'No diag':
        return 'Unknown'
    code_str = str(code).upper()
    for category, codes in diagnosis_mapping.items():
        if any(code_str.startswith(c) for c in codes):
            return category
    return 'Other'

# Apply the mapping function to diagnosis columns
diabetic_data['diag_1'] = diabetic_data['diag_1'].apply(map_diagnosis)
diabetic_data['diag_2'] = diabetic_data['diag_2'].apply(map_diagnosis)
diabetic_data['diag_3'] = diabetic_data['diag_3'].apply(map_diagnosis)

In [255]:
# Create dictionaries for the mappings
admission_type_mapping = {
    1: 'Emergency',
    2: 'Urgent',
    3: 'Elective',
    4: 'Newborn',
    5: 'Not Available',
    6: 'NULL',
    7: 'Trauma Center',
    8: 'Not Mapped'
}

discharge_disposition_mapping = {
    1: 'Discharged to home',
    2: 'Discharged/transferred to another short term hospital',
    3: 'Discharged/transferred to SNF',
    4: 'Discharged/transferred to ICF',
    5: 'Discharged/transferred to another type of inpatient care institution',
    6: 'Discharged/transferred to home with home health service',
    7: 'Left AMA',
    8: 'Discharged/transferred to home under care of Home IV provider',
    9: 'Admitted as an inpatient to this hospital',
    10: 'Neonate discharged to another hospital for neonatal aftercare',
    12: 'Still patient or expected to return for outpatient services',
    15: 'Discharged/transferred within this institution to Medicare approved swing bed',
    16: 'Discharged/transferred/referred another institution for outpatient services',
    17: 'Discharged/transferred/referred to this institution for outpatient services',
    18: 'NULL',
    22: 'Discharged/transferred to another rehab fac including rehab units of a hospital',
    23: 'Discharged/transferred to a long term care hospital',
    24: 'Discharged/transferred to a nursing facility certified under Medicaid but not certified under Medicare',
    25: 'Not Mapped',
    26: 'Unknown/Invalid',
    27: 'Discharged/transferred to a federal health care facility',
    28: 'Discharged/transferred/referred to a psychiatric hospital of psychiatric distinct part unit of a hospital',
    29: 'Discharged/transferred to a Critical Access Hospital (CAH)',
    30: 'Discharged/transferred to another Type of Health Care Institution not Defined Elsewhere'
}

admission_source_mapping = {
    1: 'Physician Referral',
    2: 'Clinic Referral',
    3: 'HMO Referral',
    4: 'Transfer from a hospital',
    5: 'Transfer from a Skilled Nursing Facility (SNF)',
    6: 'Transfer from another health care facility',
    7: 'Emergency Room',
    8: 'Court/Law Enforcement',
    9: 'Not Available',
    10: 'Transfer from critical access hospital',
    11: 'Normal Delivery',
    12: 'Premature Delivery',
    13: 'Sick Baby',
    14: 'Extramural Birth',
    15: 'Not Available',
    17: 'NULL',
    18: 'Transfer From Another Home Health Agency',
    19: 'Readmission to Same Home Health Agency',
    20: 'Not Mapped',
    21: 'Unknown/Invalid',
    22: 'Transfer from hospital inpt/same fac reslt in a sep claim',
    23: 'Born inside this hospital',
    24: 'Born outside this hospital',
    25: 'Transfer from Ambulatory Surgery Center'
}

# Define a function to map IDs to descriptions and handle non-standard values
def map_ids(df, column, mapping):
    df[column] = df[column].map(mapping).fillna('NotSpecified')

# Apply mappings to the appropriate columns
map_ids(diabetic_data, 'admission_type_id', admission_type_mapping)
map_ids(diabetic_data, 'discharge_disposition_id', discharge_disposition_mapping)
map_ids(diabetic_data, 'admission_source_id', admission_source_mapping)

# Handle non-standard values by creating a new category 'Not Specified'
diabetic_data['admission_type_id'] = diabetic_data['admission_type_id'].replace(missing_value_indicators, 'NotSpecified')
diabetic_data['discharge_disposition_id'] = diabetic_data['discharge_disposition_id'].replace(missing_value_indicators, 'NotSpecified')
diabetic_data['admission_source_id'] = diabetic_data['admission_source_id'].replace(missing_value_indicators, 'NotSpecified')

In [256]:
# Create the 'primary_diagnosis' column from 'diag_1'
diabetic_data['primary_diagnosis'] = diabetic_data['diag_1']

nonordinal_class_columns = [
    'race', 'gender', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id', 
    'medical_specialty', 'primary_diagnosis', 'max_glu_serum', 'A1Cresult','metformin', 
    'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide', 
    'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'tolazamide', 
    'insulin','glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone',
    'metformin-rosiglitazone', 'metformin-pioglitazone'
]

# One-hot encoding for 'diag_2' and 'diag_3'
diag_2_encoded = pd.get_dummies(diabetic_data['diag_2'], prefix='secondary_diag', dtype=int)
diag_3_encoded = pd.get_dummies(diabetic_data['diag_3'], prefix='secondary_diag', dtype=int)
secondary_diag_encoded = diag_2_encoded.add(diag_3_encoded)
diabetic_data = pd.concat([diabetic_data, secondary_diag_encoded], axis=1)

# One-hot encode the remaining nonordial columns
diabetic_data = pd.get_dummies(diabetic_data, columns=nonordinal_class_columns, dtype=int)
# Drop the original diagnosis columns ('diag_1', 'diag_2', 'diag_3')
diabetic_data.drop(columns=['diag_1', 'diag_2', 'diag_3', 'diabetesMed', 'encounter_id', 'patient_nbr'], inplace=True)

# Display the first few rows of the one-hot encoded data
print(diabetic_data.head())

           age  time_in_hospital  num_lab_procedures  num_procedures  \
4267   [50-60)                 8                  77               6   
5827   [50-60)                 2                  49               1   
67608  [80-90)                 4                  68               2   
17494  [80-90)                 3                  46               0   
2270   [30-40)                 5                  49               0   

       num_medications  number_outpatient  number_emergency  number_inpatient  \
4267                33                  0                 0                 0   
5827                11                  0                 0                 0   
67608               23                  0                 0                 0   
17494               20                  0                 0                 0   
2270                 5                  0                 0                 0   

       number_diagnoses change  ... glyburide-metformin_No  \
4267              

In [257]:
# Define the mapping for the age ranges
age_mapping = {
    '[0-10)': 0,
    '[10-20)': 1,
    '[20-30)': 2,
    '[30-40)': 3,
    '[40-50)': 4,
    '[50-60)': 5,
    '[60-70)': 6,
    '[70-80)': 7,
    '[80-90)': 8,
    '[90-100)': 9
}

# Apply the mapping to the 'age' column
diabetic_data['age'] = diabetic_data['age'].map(age_mapping)

In [258]:
# Define the mapping for the change feature
change_mapping = {
    'Ch': 1,
    'No': 0
}

# Apply the mapping to the 'change' column
diabetic_data['change'] = diabetic_data['change'].map(change_mapping)

#One hot encode the readmitted column
diabetic_data = pd.concat([diabetic_data,pd.get_dummies(diabetic_data['readmitted'], prefix='readmission_label', dtype=int)], axis=1)
diabetic_data.drop(columns=['readmitted'], inplace=True)

In [259]:

# Scaling numerical features
numerical_features = diabetic_data.select_dtypes(include=['int64', 'float64']).columns
scaler = MinMaxScaler()
diabetic_data[numerical_features] = scaler.fit_transform(diabetic_data[numerical_features])
diabetic_data.to_csv('diabetic_data_final.csv', index=False)


In [260]:
import joblib
# Save the scaler for future use
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']